In [24]:
import os
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from time import sleep

In [25]:
os.environ['PATH'] += r"/usr/local/bin/"
driver = webdriver.Chrome()
driver.maximize_window()
sleep(5)

In [26]:
def get_data(text):
    reg_template = {"Area":'Diện tích:\s*(\d+)',
                "Room":'Phòng ngủ:\s*(\d+)',
                "WC":'Phòng WC:\s*(\d+)',
                "Location":'Địa chỉ:\s*(.+)'}
    list_params = []
    for i,name in enumerate(reg_template):
        x = re.search(r'{}'.format(reg_template[name]),text)
        if x:
            list_params.append(x.group(1))
        else:
            list_params.append(np.nan)
    return list_params

In [27]:
base_url = "https://batdongsan.vn/ban-nha/"
report =    pd.DataFrame(columns=['Title', 'Price', 'Links',\
            'Diện tích (m2)', 'Số phòng ngủ','Số phòng WC', 'Địa chỉ'\
            ,'Description'])

page = 2

for i in range(1):
    url = base_url + "p{}".format(i+1)
    print("Loading")
    driver.get(url)
    print("Loading")

    driver.implicitly_wait(1)
    try:
        print("Loading")
        elements = driver.find_elements('css selector','.uk-width-medium-7-10 .name [href]')
        elements_price = driver.find_elements('css selector','.price')
    except:
        continue
    
    title = [element.text for element in elements]

    links = [element.get_attribute('href') for element in elements]

    price = [element.text for element in elements_price]
    
    df1 = pd.DataFrame(zip(title,price,links),columns=['Title','Price','Links'])
    
    data = []
    
    n = len(df1)

    for j in range(n):
        try:
            driver.get(links[j])
            params = driver.find_element('css selector',".param")
            content = driver.find_element('css selector',".content")
        except:
            df1.drop(j,axis=0,inplace=True)
            continue
        params_ = get_data(params.text)
        params_.append(content.text)
        data.append(params_)

    tmp = np.vstack([item for item in data])
    df2 = pd.DataFrame(tmp,columns=['Diện tích (m2)','Số phòng ngủ','Số phòng WC','Địa chỉ', 'Description'])
    df1 = pd.concat([df1,df2],axis=1)
    report = pd.concat([report,df1],axis=0)
    report.set_index(np.arange(len(report)),inplace=True)
    report.to_csv('output.csv',sep="\t",index=False)

report

Loading
Loading
Loading
